<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/04_ejercicio_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
#!pip install polars

In [1]:
import polars as pl
import random
import string


# Generacion de Datos

In [2]:
# Funciones auxiliares para generar datos
def random_string(length=5):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_data(n=1000):
    data = {
        "id_venta": list(range(1, n + 1)),
        "producto": [random_string() for _ in range(n)],
        "cantidad": [random.randint(1, 20) for _ in range(n)],
        "precio_unitario": [random.uniform(10, 100) for _ in range(n)],
        "fecha_venta": [pl.date(random.randint(2010, 2022), random.randint(1, 12), random.randint(1, 28)) for _ in range(n)],
        "cliente_id": [random.randint(1000, 5000) for _ in range(n)]
    }
    return pl.DataFrame(data)

# Generar el DataFrame
df = generate_data()
df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,cliente_id
i64,str,i64,f64,object,i64
1,"""bwutf""",12,21.908184,"2011-12-23 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2891
2,"""purcj""",13,56.930169,"2021-02-12 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4646
3,"""tutdx""",10,20.278388,"2012-03-05 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1087
4,"""runqb""",12,38.246668,"2012-03-08 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",3397
5,"""htvkr""",12,57.738777,"2022-06-05 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1256


In [3]:
df = df.lazy()
df

# Limpieza de Datos

Utilizando Polars en modo Lazy, realice las siguientes operaciones de limpieza y manipulación de datos:

    1. `Filtrar` las ventas con cantidades menores a 5.
    2. `Agregar` una nueva columna llamada total_venta, que sea el resultado de multiplicar cantidad por precio_unitario.
    3. `Eliminar` la columna precio_unitario después de crear total_venta.
    4. `Agrupar` los datos por producto y calcular el total vendido y la cantidad total vendida por producto.
    5. `Ordenar` los resultados por el total vendido en orden descendente.

Todo esto lo debes de realizar en una sola linea de codigo

In [4]:

lazy_df = df.lazy()

# Realizar las operaciones de limpieza y manipulación
resultado = (
    lazy_df
    .filter(pl.col("cantidad") >= 5)
    .with_columns((pl.col("cantidad") * pl.col("precio_unitario")).alias("total_venta"))
   
    .drop("precio_unitario")
    .group_by("producto")
    .agg([
        pl.sum("total_venta").alias("total_vendido"),
        pl.sum("cantidad").alias("cantidad_total_vendida"),
        
    ])
    .sort("total_vendido", descending = True)
)

  

# Ejecutar y mostrar los resultados
resultado_df = resultado.collect()
print(resultado_df)


shape: (806, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ sgdvw    ┆ 1951.875877   ┆ 20                     │
│ vmajn    ┆ 1893.229615   ┆ 20                     │
│ xzaqq    ┆ 1892.459543   ┆ 19                     │
│ fauam    ┆ 1885.251115   ┆ 19                     │
│ mvhbq    ┆ 1852.555393   ┆ 19                     │
│ …        ┆ …             ┆ …                      │
│ lkkku    ┆ 64.484945     ┆ 5                      │
│ lflrn    ┆ 63.22095      ┆ 5                      │
│ ewdkp    ┆ 58.184666     ┆ 5                      │
│ enwsx    ┆ 54.701273     ┆ 5                      │
│ lzjsx    ┆ 52.810163     ┆ 5                      │
└──────────┴───────────────┴────────────────────────┘


# Guarda el resultado en un archivo `parquet`

Guarda el resultado en un archivo parquet que se llame `resultados.parquet`

In [5]:
# Tu codigo aqui
resultado_df = resultado.collect()

# Guardar el DataFrame en un archivo Parquet
resultado_df.write_parquet("resultado.parquet")

# Lee Parquet

Crea codigo para leer en polars el archivo parquet

In [6]:
# Tu codigo aqui
df = pl.read_parquet("resultado.parquet")

# Mostrar las primeras filas del DataFrame
print(df.head())

shape: (5, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ sgdvw    ┆ 1951.875877   ┆ 20                     │
│ vmajn    ┆ 1893.229615   ┆ 20                     │
│ xzaqq    ┆ 1892.459543   ┆ 19                     │
│ fauam    ┆ 1885.251115   ┆ 19                     │
│ mvhbq    ┆ 1852.555393   ┆ 19                     │
└──────────┴───────────────┴────────────────────────┘
